# Software Environment and Cloud Computing

[![Dependency](fig/dependency.png)](https://xkcd.com/2347/)

In modern computational science, running code is only half the
challenge.
The other half is managing the software environment it depends on.
Different projects may require different compilers, libraries, or
Python packages, and the ability to reproduce results across HPC
systems, laptops, and cloud platforms is important.

This week, we will learn practical tools for creating and managing
software environments:
* Package managers (e.g., `apt-get`) for installing software at the
  system level
* Building from source when precompiled packages are unavailable or
  when you do not have root access to the system.
* HPC `module` for loading different versions of software on a shared
  system.
* Python virtual environments for dependency isolation
* Containers (i.e., Docker) for reproducibility and portability

## Software Management Basics

On Unix/Linux systems, most software is installed through package
managers, which automatically handle downloading, installing, and
configuring programs and their dependencies.

Common package managers include:
* Debian/Ubuntu: [`apt`](https://wiki.debian.org/Apt) (or the older `apt-get`)
* Red Hat/Fedora/CentOS: [`yum`](https://fedoraproject.org/wiki/Yum) or [`dnf`](https://docs.fedoraproject.org/en-US/quick-docs/dnf/)
* Arch Linux: [`pacman`](https://wiki.archlinux.org/title/Pacman)
* macOS: [`brew`](https://brew.sh/) (Homebrew) and [`port`](https://www.macports.org/) (MacPorts)

[![sudo](fig/sandwich.png)](https://xkcd.com/149/)

By default, package managers install software system-wide, which
requires root (administrator) privileges.
On personal machines, this is often done by prefixing commands with
`sudo` (i.e., "superuser do").

On shared systems such as HPC clusters, users typically do not have
root access, so alternative approaches are used (modules, virtual
environments, containers).

Here, we use Docker to create a sandbox environment where you can
safely practice system-level package management commands.
We will cover Docker in more detail later, but for now, assuming
Docker is installed, you can start a sandbox with:
```bash
docker run -it --rm debian:forky-slim
```

Inside your Docker container, update the package list (always do
this first):
```bash
apt update
```

Install a simple system utility (e.g., `htop`) and a scientific
library (e.g., GNU Scientific Library `gsl`):
```bash
apt install -y htop
apt install -y libgsl-dev
```

You may now verify the installation:
```bash
htop                 # should start the process viewer
dpkg -l | grep gsl   # list installed gsl package
```

## Building from Source

Not all software is available in package repositories or is prepared
in a way that you want it.
Sometimes the version provided by the package manager is outdated, or
the software is not packaged at all.
Sometimes you may want to enable some special settings or install a
package in non-standard locations.
In these cases, you can download the source code and compile it
yourself.

[![Virus on Linux](fig/virus.png)](https://www.gnu.org/fun/jokes/evilmalware.en.html)

This process usually follows steps:
1. **Configure** the build system (check dependencies, set options).
2. **Compile** the source code.
3. **Check** the resulting binaries.
4. **Install** the resulting binaries.

Start a sandbox in Docker with gcc; then download GSL
```bash
docker run -it --rm gcc
mkdir /src && cd /src
wget https://ftp.wayne.edu/gnu/gsl/gsl-2.8.tar.gz
tar -xvzf gsl-2.8.tar.gz && cd gsl-2.8
```

Configure, build, check, and install:
```bash
./configure
make
make check
make install
```

Optionally, you may test it by compiling a small program:
```c
/* Save as "test.c" and then compile by `gcc test.c -o test -lgsl` */
#include <stdio.h>
#include <gsl/gsl_sf_bessel.h>

int main() {
    double x = 2.4048; /* first root of J0 */
    double y = gsl_sf_bessel_J0(x);
    printf("J0(%g) = %.18e\n", x, y);
    return 0;
}
```

In [ ]:
# HANDSON: how do you get the above source code in a Docker container?


In [ ]:
# HANDSON: suppose that you don't have root access to a machine.
#          Let's install GSL to "/home/me/.local/" instead of the
#          standard location "/usr/local/".
#          How do you do it?
#          Hint: run `/configure --help` to see all the different
#          options.


[![Python Environment](fig/python_environment.png)](https://xkcd.com/1987/)